In [92]:
import json
import math
import csv
import os

def calculate_mean(values):
    return sum(values) / len(values)

def calculate_standard_deviation(values):
    mean = calculate_mean(values)
    variance = sum((x - mean) ** 2 for x in values) / len(values)
    return math.sqrt(variance)

def calculate_rmse(values):
    return math.sqrt(sum(x ** 2 for x in values) / len(values))

def process_data(calibration_file, experiment_file):
    # Load calibration data
    with open(calibration_file, 'r') as file:
        calibration_data = json.load(file)

    # Load experiment data
    with open(experiment_file, 'r') as file:
        experiment_data = json.load(file)

    # Extract calibration metrics
    accuracy = calibration_data.get('accuracy', 0)
    smallest_distance = calibration_data.get('smallestDistance', 0)
    largest_distance = calibration_data.get('largestDistance', 0)
    regression_type = calibration_data.get('regressionType', '')
    num_calibration_clicks = calibration_data.get('numCalibrationClicks', 0)
    distances = calibration_data.get('distance50', [])
    mean_distance = calculate_mean(distances)
    standard_deviation_distance = calculate_standard_deviation(distances)
    rmse = calculate_rmse(distances)

    # Extract experiment metrics
    total_time_taken = sum(page_data['time_taken_seconds'] for page_data in experiment_data['data'].values())
    total_webpages_visited = len(experiment_data['data'])
    avg_time_per_webpage = total_time_taken / total_webpages_visited if total_webpages_visited > 0 else 0
    total_unique_tabs_visited = len(experiment_data['data'])

    # Calculate average sample rate
    total_sample_rate = sum(page_data['average_sample_rate'] for page_data in experiment_data['data'].values())
    avg_sample_rate = total_sample_rate / total_webpages_visited if total_webpages_visited > 0 else 0

    # Participant number input
    participant_number = input("Enter Participant Number: ")

    # Banking website selection
    banking_website = select_banking_website_used()

    # Banking website selection
    task_attempted = select_task_attempted()

    # Write data to CSV
    csv_file_path = 'experiment_metrics.csv'
    with open(csv_file_path, 'w', newline='') as csvfile:
        fieldnames = ['Participant Number', 'Banking Website', 'Task Attempted', 'Accuracy', 'Smallest Distance', 'Largest Distance', 'Regression Type',
                      'Number of Calibration Clicks', 'Mean Distance', 'Standard Deviation of Distance',
                      'RMSE', 'Total Time Taken', 'Total Number of Webpages Visited',
                      'Average Time Spent per Webpage', 'Total Unique Tabs Visited', 'Average Sample Rate']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        writer.writerow({'Participant Number': participant_number,
                         'Banking Website': banking_website,
                         'Task Attempted': task_attempted,
                         'Accuracy': accuracy,
                         'Smallest Distance': smallest_distance,
                         'Largest Distance': largest_distance,
                         'Regression Type': regression_type,
                         'Number of Calibration Clicks': num_calibration_clicks,
                         'Mean Distance': mean_distance,
                         'Standard Deviation of Distance': standard_deviation_distance,
                         'RMSE': rmse,
                         'Total Time Taken': total_time_taken,
                         'Total Number of Webpages Visited': total_webpages_visited,
                         'Average Time Spent per Webpage': avg_time_per_webpage,
                         'Total Unique Tabs Visited': total_unique_tabs_visited,
                         'Average Sample Rate': avg_sample_rate})

    print(f"Data has been exported to '{csv_file_path}'.")

def select_banking_website_used():
    print("Select Banking Website Used:")
    print("1. HSBC")
    print("2. Lombard")
    print("3. BNF")
    while True:
        choice = input("Enter option number: ")
        if choice in ['1', '2', '3']:
            if choice == '1':
                return "HSBC"
            elif choice == '2':
                return "Lombard"
            elif choice == '3':
                return "BNF"
        else:
            print("Invalid option. Please enter a valid option number.")

def select_task_attempted():
    print("Select Task Attempted:")
    print("1. Personal Loan")
    print("2. Fixed Term Deposit")
    while True:
        choice = input("Enter option number: ")
        if choice in ['1', '2']:
            if choice == '1':
                return "Personal Loan"
            elif choice == '2':
                return "Fixed Term Deposit"
        else:
            print("Invalid option. Please enter a valid option number.")

# Directory paths
calibration_directory = "calibration"
experiment_directory = "data"

# Find calibration JSON file
calibration_files = [f for f in os.listdir(calibration_directory) if f.endswith('.json')]
if len(calibration_files) == 1:
    calibration_file = os.path.join(calibration_directory, calibration_files[0])
else:
    print("Error: Calibration directory should contain exactly one JSON file.")
    exit()

# Find experiment JSON file
experiment_files = [f for f in os.listdir(experiment_directory) if f.endswith('.json')]
if len(experiment_files) == 1:
    experiment_file = os.path.join(experiment_directory, experiment_files[0])
else:
    print("Error: Experiment directory should contain exactly one JSON file.")
    exit()

# Process data and export to CSV
process_data(calibration_file, experiment_file)

Select Banking Website Used:
1. HSBC
2. Lombard
3. BNF
Select Task Attempted:
1. Personal Loan
2. Fixed Term Deposit
Data has been exported to 'experiment_metrics.csv'.
